In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

! rm -r vertebral.mat wine.mat pima.mat ionosphere.mat wbc.mat glass.mat vowels.mat letter.mat 
! wget https://www.dropbox.com/s/5kuqb387sgvwmrb/vertebral.mat
! wget https://www.dropbox.com/s/uvjaudt2uto7zal/wine.mat
! wget https://www.dropbox.com/s/mvlwu7p0nyk2a2r/pima.mat
! wget https://www.dropbox.com/s/lpn4z73fico4uup/ionosphere.mat
! wget https://www.dropbox.com/s/ebz9v9kdnvykzcb/wbc.mat
! wget https://www.dropbox.com/s/iq3hjxw77gpbl7u/glass.mat
! wget https://www.dropbox.com/s/pa26odoq6atq9vx/vowels.mat
! wget https://www.dropbox.com/s/rt9i95h9jywrtiy/letter.mat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 52.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-yk8imaju
  Running command git clone --filter=blob:none --quiet https://github.com/p

In [ ]:
import math
import torch
import random
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

import torch_geometric.nn.conv as gnn

from sklearn.metrics import auc
from torch_geometric.data import Data

In [ ]:
device = torch.device('cpu')

In [ ]:
def cosine_similarity(a, b):
    a = a / np.sum(a ** 2) ** 0.5
    b = b / np.sum(b ** 2) ** 0.5
    return np.sum(a * b)
    # return 1/(1+euclidean_distance(a, b))

In [ ]:
def euclidean_distance(a, b):
    dist = np.sum((a - b) ** 2) ** 0.5
    return dist

In [ ]:
data = sio.loadmat('wbc.mat')

x = data['X'].astype(np.float32)
y = data['y'].astype(np.float32)

# x = x - np.mean(x, axis = 0)
# x = x / np.std(x, axis = 0)

ind = list(range(x.shape[0]))
random.shuffle(ind)
x, y = x[ind], y[ind]

print(x.shape, y.shape)

(378, 30) (378, 1)


In [ ]:
# Edge selection based on euclidean distance
# k = int(x.shape[0] // 40)
k = 10
edge_index = []
edge_weights = []
for i in range(x.shape[0]):
    neighbours = []
    for j in range(x.shape[0]):
        similarity =  cosine_similarity(x[i], x[j])
        neighbours.append([similarity, j])
    neighbours.sort(reverse = True)
    for ind in range(1, k + 1):
        j = neighbours[ind][1]
        similarity = neighbours[ind][0]
        edge_index.append([j, i])
        edge_weights.append(similarity)

edge_index = np.array(edge_index).astype(np.int64)
edge_index = edge_index.T
print(edge_index.shape)

edge_weights = np.array(edge_weights).astype(np.float32)
print(edge_weights.shape)


(2, 3780)
(3780,)


In [ ]:
# PyTorch Graph creation
graph = Data(x = torch.tensor(x), edge_index = torch.tensor(edge_index), edge_attr=torch.tensor(edge_weights))
graph = graph.to(device)
display(graph)

Data(x=[378, 30], edge_index=[2, 3780], edge_attr=[3780])

In [ ]:
# Graph based model
class GAE(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.p = 0.5
        self.bn = torch.nn.BatchNorm1d(dim_in)
        self.graph_conv1 = gnn.GraphConv(dim_h, dim_h) 

        # in_channels: The number of input channels for the graph convolution layer. 
        # This is typically equal to the number of features or attributes for each node in the graph.

        # out_channels: The number of output channels for the graph convolution layer. 
        # This determines the number of features or attributes that will be output for each node in the graph.

        self.graph_conv2 = gnn.GraphConv(dim_h, dim_h)
        self.input = torch.nn.Linear(dim_in, dim_h)
        self.output = torch.nn.Linear(dim_h, dim_out)

    def forward(self, data):
        h = data.x
        h = self.bn(h)
        h = self.input(h)
        h = torch.relu(h)
        h = self.graph_conv1(h, data.edge_index, data.edge_attr)
        h = torch.relu(h)
        layer_1_output = h
        h = torch.nn.functional.dropout(h, p = self.p, training = self.training)
        h = self.graph_conv2(h, data.edge_index, data.edge_attr)
        h = torch.relu(h)
        layer_2_output = h
        h = (layer_1_output + layer_2_output) * 0.5
        h = self.output(h)
        return h

In [ ]:
# Loss function
def loss_function(embeddings, graph):
    loss = embeddings - graph.x
    loss = torch.sum(loss ** 2, axis = 1)
    loss = torch.mean(loss ** 0.5)
    return loss

In [ ]:
# AUC evaluation
def evaluate(x, x_emb, y, plot = False):
    ranking = []
    for i in range(x.shape[0]):
        distance = euclidean_distance(x[i], x_emb[i])
        ranking.append([distance, y[i][0]])
    ranking.sort(reverse = True)
    tpr, fpr = [], []
    positive = np.sum(y)
    negative = x.shape[0] - np.sum(y)
    for outlier_count in range(1, x.shape[0] + 1):
        tp, fp = 0, 0
        for i in range(outlier_count):
            tp += ranking[i][-1]
            fp += 1 - ranking[i][-1]
        tpr.append(tp / positive)
        fpr.append(fp / negative)
    roc_auc = auc(fpr, tpr)
    if plot:
        plt.plot(fpr, tpr)
        plt.plot(fpr, fpr)
        plt.grid()
        plt.title('ROC AUC: %.3f' % (roc_auc))
        plt.show()
    else:
        return roc_auc

In [ ]:
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

best_roc = 0
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()  #model.train() is a method in PyTorch that sets the model in training mode. 
    #This means that the model will behave differently during training compared to evaluation or inference.
    optimizer.zero_grad() #optimizer.zero_grad() is a method in PyTorch that sets the gradients of all parameters in the optimizer to zero. 
    #This is typically done before computing the gradients for a new batch of data during the training process of a neural network
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()#optimizer.step() is a method in PyTorch that is used to update 
    #the parameters of a neural network during the training process using an optimization algorithm.
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()  #The .detach() method is commonly used to extract a tensor from a larger computation graph
            # and use it in a separate calculation or for checkpointing during training. 
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 0.98 	 ROC AUC: 0.97
Epoch: 20 	 Loss: 0.58 	 ROC AUC: 0.94
Epoch: 30 	 Loss: 0.52 	 ROC AUC: 0.93
Epoch: 40 	 Loss: 0.48 	 ROC AUC: 0.92
Epoch: 50 	 Loss: 0.43 	 ROC AUC: 0.90
Epoch: 60 	 Loss: 0.38 	 ROC AUC: 0.90
Epoch: 70 	 Loss: 0.36 	 ROC AUC: 0.90
Epoch: 80 	 Loss: 0.34 	 ROC AUC: 0.89
Epoch: 90 	 Loss: 0.32 	 ROC AUC: 0.90
Epoch: 100 	 Loss: 0.32 	 ROC AUC: 0.89
Epoch: 110 	 Loss: 0.30 	 ROC AUC: 0.89
Epoch: 120 	 Loss: 0.30 	 ROC AUC: 0.90
Epoch: 130 	 Loss: 0.29 	 ROC AUC: 0.89
Epoch: 140 	 Loss: 0.28 	 ROC AUC: 0.89
Epoch: 150 	 Loss: 0.28 	 ROC AUC: 0.89
Epoch: 160 	 Loss: 0.27 	 ROC AUC: 0.89
Epoch: 170 	 Loss: 0.27 	 ROC AUC: 0.88
Epoch: 180 	 Loss: 0.26 	 ROC AUC: 0.89
Epoch: 190 	 Loss: 0.25 	 ROC AUC: 0.89
Epoch: 200 	 Loss: 0.25 	 ROC AUC: 0.88
Epoch: 210 	 Loss: 0.24 	 ROC AUC: 0.87
Epoch: 220 	 Loss: 0.24 	 ROC AUC: 0.88
Epoch: 230 	 Loss: 0.24 	 ROC AUC: 0.87
Epoch: 240 	 Loss: 0.23 	 ROC AUC: 0.86
Epoch: 250 	 Loss: 0.23 	 ROC AUC: 0.86
Epoch: 26

## Dataset Wise Records

### Dataset: wbc

In [ ]:
# wbc :: k=10 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

best_roc = 0
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 1.11 	 ROC AUC: 0.98
Epoch: 20 	 Loss: 0.66 	 ROC AUC: 0.98
Epoch: 30 	 Loss: 0.55 	 ROC AUC: 0.95
Epoch: 40 	 Loss: 0.50 	 ROC AUC: 0.93
Epoch: 50 	 Loss: 0.46 	 ROC AUC: 0.90
Epoch: 60 	 Loss: 0.43 	 ROC AUC: 0.87
Epoch: 70 	 Loss: 0.41 	 ROC AUC: 0.87
Epoch: 80 	 Loss: 0.38 	 ROC AUC: 0.87
Epoch: 90 	 Loss: 0.37 	 ROC AUC: 0.86
Epoch: 100 	 Loss: 0.35 	 ROC AUC: 0.83
Epoch: 110 	 Loss: 0.34 	 ROC AUC: 0.87
Epoch: 120 	 Loss: 0.33 	 ROC AUC: 0.88
Epoch: 130 	 Loss: 0.31 	 ROC AUC: 0.86
Epoch: 140 	 Loss: 0.31 	 ROC AUC: 0.87
Epoch: 150 	 Loss: 0.30 	 ROC AUC: 0.89
Epoch: 160 	 Loss: 0.29 	 ROC AUC: 0.88
Epoch: 170 	 Loss: 0.29 	 ROC AUC: 0.84
Epoch: 180 	 Loss: 0.28 	 ROC AUC: 0.85
Epoch: 190 	 Loss: 0.27 	 ROC AUC: 0.84
Epoch: 200 	 Loss: 0.27 	 ROC AUC: 0.86
Epoch: 210 	 Loss: 0.27 	 ROC AUC: 0.84
Epoch: 220 	 Loss: 0.26 	 ROC AUC: 0.85
Epoch: 230 	 Loss: 0.25 	 ROC AUC: 0.85
Epoch: 240 	 Loss: 0.25 	 ROC AUC: 0.86
Epoch: 250 	 Loss: 0.24 	 ROC AUC: 0.84
Epoch: 26

### Dataset: wine

In [ ]:
# wine :: k=10 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

best_roc = 0
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 594.55 	 ROC AUC: 1.00
Epoch: 20 	 Loss: 309.24 	 ROC AUC: 1.00
Epoch: 30 	 Loss: 183.57 	 ROC AUC: 1.00
Epoch: 40 	 Loss: 143.84 	 ROC AUC: 1.00
Epoch: 50 	 Loss: 107.23 	 ROC AUC: 0.85
Epoch: 60 	 Loss: 84.22 	 ROC AUC: 0.92
Epoch: 70 	 Loss: 79.54 	 ROC AUC: 0.92
Epoch: 80 	 Loss: 79.52 	 ROC AUC: 0.84
Epoch: 90 	 Loss: 90.30 	 ROC AUC: 0.84
Epoch: 100 	 Loss: 115.33 	 ROC AUC: 0.84
Epoch: 110 	 Loss: 84.34 	 ROC AUC: 0.87
Epoch: 120 	 Loss: 80.58 	 ROC AUC: 0.87
Epoch: 130 	 Loss: 85.29 	 ROC AUC: 0.79
Epoch: 140 	 Loss: 73.47 	 ROC AUC: 0.88
Epoch: 150 	 Loss: 75.22 	 ROC AUC: 0.78
Epoch: 160 	 Loss: 78.13 	 ROC AUC: 0.73
Epoch: 170 	 Loss: 72.66 	 ROC AUC: 0.83
Epoch: 180 	 Loss: 71.00 	 ROC AUC: 0.77
Epoch: 190 	 Loss: 72.99 	 ROC AUC: 0.85
Epoch: 200 	 Loss: 71.91 	 ROC AUC: 0.71
Epoch: 210 	 Loss: 78.12 	 ROC AUC: 0.77
Epoch: 220 	 Loss: 60.06 	 ROC AUC: 0.84
Epoch: 230 	 Loss: 64.81 	 ROC AUC: 0.88
Epoch: 240 	 Loss: 71.01 	 ROC AUC: 0.91
Epoch: 250 	 Loss: 

### Dataset: pima

In [ ]:
# pima :: k=20 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

best_roc = 0
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 172.89 	 ROC AUC: 0.77
Epoch: 20 	 Loss: 135.48 	 ROC AUC: 0.74
Epoch: 30 	 Loss: 108.29 	 ROC AUC: 0.68
Epoch: 40 	 Loss: 93.09 	 ROC AUC: 0.64
Epoch: 50 	 Loss: 85.32 	 ROC AUC: 0.67
Epoch: 60 	 Loss: 78.36 	 ROC AUC: 0.71
Epoch: 70 	 Loss: 70.04 	 ROC AUC: 0.67
Epoch: 80 	 Loss: 61.86 	 ROC AUC: 0.67
Epoch: 90 	 Loss: 54.20 	 ROC AUC: 0.63
Epoch: 100 	 Loss: 50.90 	 ROC AUC: 0.64
Epoch: 110 	 Loss: 45.50 	 ROC AUC: 0.62
Epoch: 120 	 Loss: 42.27 	 ROC AUC: 0.61
Epoch: 130 	 Loss: 44.64 	 ROC AUC: 0.62
Epoch: 140 	 Loss: 43.26 	 ROC AUC: 0.61
Epoch: 150 	 Loss: 43.25 	 ROC AUC: 0.61
Epoch: 160 	 Loss: 40.63 	 ROC AUC: 0.62
Epoch: 170 	 Loss: 41.32 	 ROC AUC: 0.61
Epoch: 180 	 Loss: 40.35 	 ROC AUC: 0.63
Epoch: 190 	 Loss: 39.34 	 ROC AUC: 0.66
Epoch: 200 	 Loss: 40.40 	 ROC AUC: 0.67
Epoch: 210 	 Loss: 38.95 	 ROC AUC: 0.66
Epoch: 220 	 Loss: 39.32 	 ROC AUC: 0.62
Epoch: 230 	 Loss: 40.01 	 ROC AUC: 0.57
Epoch: 240 	 Loss: 40.64 	 ROC AUC: 0.60
Epoch: 250 	 Loss: 39.

### Dataset: glass

In [ ]:
# glass :: k=20 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

best_roc = 0
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 24.62 	 ROC AUC: 0.35
Epoch: 20 	 Loss: 13.47 	 ROC AUC: 0.62
Epoch: 30 	 Loss: 9.84 	 ROC AUC: 0.43
Epoch: 40 	 Loss: 10.16 	 ROC AUC: 0.65
Epoch: 50 	 Loss: 9.67 	 ROC AUC: 0.83
Epoch: 60 	 Loss: 5.98 	 ROC AUC: 0.80
Epoch: 70 	 Loss: 7.86 	 ROC AUC: 0.88
Epoch: 80 	 Loss: 9.34 	 ROC AUC: 0.83
Epoch: 90 	 Loss: 10.16 	 ROC AUC: 0.85
Epoch: 100 	 Loss: 10.97 	 ROC AUC: 0.96
Epoch: 110 	 Loss: 9.27 	 ROC AUC: 0.84
Epoch: 120 	 Loss: 5.71 	 ROC AUC: 0.59
Epoch: 130 	 Loss: 9.45 	 ROC AUC: 0.94
Epoch: 140 	 Loss: 7.65 	 ROC AUC: 0.95
Epoch: 150 	 Loss: 7.68 	 ROC AUC: 0.94
Epoch: 160 	 Loss: 7.22 	 ROC AUC: 0.83
Epoch: 170 	 Loss: 5.13 	 ROC AUC: 0.82
Epoch: 180 	 Loss: 6.17 	 ROC AUC: 0.79
Epoch: 190 	 Loss: 6.03 	 ROC AUC: 0.82
Epoch: 200 	 Loss: 6.33 	 ROC AUC: 0.83
Epoch: 210 	 Loss: 4.74 	 ROC AUC: 0.71
Epoch: 220 	 Loss: 6.14 	 ROC AUC: 0.75
Epoch: 230 	 Loss: 9.74 	 ROC AUC: 0.79
Epoch: 240 	 Loss: 6.42 	 ROC AUC: 0.84
Epoch: 250 	 Loss: 7.96 	 ROC AUC: 0.78
Epoc

### Dataset: vertebral

In [ ]:
# vertebral :: k=10 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

best_roc = 0
epochs = 200
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 148.37 	 ROC AUC: 0.63
Epoch: 20 	 Loss: 102.44 	 ROC AUC: 0.62
Epoch: 30 	 Loss: 65.82 	 ROC AUC: 0.84
Epoch: 40 	 Loss: 48.65 	 ROC AUC: 0.33
Epoch: 50 	 Loss: 42.38 	 ROC AUC: 0.64
Epoch: 60 	 Loss: 41.17 	 ROC AUC: 0.51
Epoch: 70 	 Loss: 35.17 	 ROC AUC: 0.38
Epoch: 80 	 Loss: 34.45 	 ROC AUC: 0.43
Epoch: 90 	 Loss: 36.58 	 ROC AUC: 0.49
Epoch: 100 	 Loss: 36.43 	 ROC AUC: 0.39
Epoch: 110 	 Loss: 34.24 	 ROC AUC: 0.56
Epoch: 120 	 Loss: 34.40 	 ROC AUC: 0.43
Epoch: 130 	 Loss: 35.06 	 ROC AUC: 0.40
Epoch: 140 	 Loss: 34.74 	 ROC AUC: 0.64
Epoch: 150 	 Loss: 31.72 	 ROC AUC: 0.31
Epoch: 160 	 Loss: 36.95 	 ROC AUC: 0.37
Epoch: 170 	 Loss: 33.85 	 ROC AUC: 0.39
Epoch: 180 	 Loss: 32.86 	 ROC AUC: 0.56
Epoch: 190 	 Loss: 35.47 	 ROC AUC: 0.33
Epoch: 200 	 Loss: 32.95 	 ROC AUC: 0.39

ROC: 0.84


### Dataset: ionosphere

In [ ]:
# ionosphere :: k=10 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

best_roc = 0
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 2.90 	 ROC AUC: 0.73
Epoch: 20 	 Loss: 2.38 	 ROC AUC: 0.85
Epoch: 30 	 Loss: 2.07 	 ROC AUC: 0.88
Epoch: 40 	 Loss: 1.92 	 ROC AUC: 0.91
Epoch: 50 	 Loss: 1.71 	 ROC AUC: 0.94
Epoch: 60 	 Loss: 1.69 	 ROC AUC: 0.95
Epoch: 70 	 Loss: 1.60 	 ROC AUC: 0.95
Epoch: 80 	 Loss: 1.57 	 ROC AUC: 0.95
Epoch: 90 	 Loss: 1.64 	 ROC AUC: 0.95
Epoch: 100 	 Loss: 1.54 	 ROC AUC: 0.95
Epoch: 110 	 Loss: 1.49 	 ROC AUC: 0.95
Epoch: 120 	 Loss: 1.51 	 ROC AUC: 0.95
Epoch: 130 	 Loss: 1.46 	 ROC AUC: 0.95
Epoch: 140 	 Loss: 1.43 	 ROC AUC: 0.95
Epoch: 150 	 Loss: 1.41 	 ROC AUC: 0.95
Epoch: 160 	 Loss: 1.39 	 ROC AUC: 0.95
Epoch: 170 	 Loss: 1.37 	 ROC AUC: 0.96
Epoch: 180 	 Loss: 1.35 	 ROC AUC: 0.95
Epoch: 190 	 Loss: 1.33 	 ROC AUC: 0.96
Epoch: 200 	 Loss: 1.32 	 ROC AUC: 0.96
Epoch: 210 	 Loss: 1.29 	 ROC AUC: 0.96
Epoch: 220 	 Loss: 1.28 	 ROC AUC: 0.96
Epoch: 230 	 Loss: 1.26 	 ROC AUC: 0.96
Epoch: 240 	 Loss: 1.24 	 ROC AUC: 0.96
Epoch: 250 	 Loss: 1.23 	 ROC AUC: 0.96
Epoch: 26

### Dataset: vowels

In [ ]:
# vowels :: k=40 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

best_roc = 0
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 3.85 	 ROC AUC: 0.57
Epoch: 20 	 Loss: 3.41 	 ROC AUC: 0.62
Epoch: 30 	 Loss: 3.36 	 ROC AUC: 0.63
Epoch: 40 	 Loss: 3.29 	 ROC AUC: 0.64
Epoch: 50 	 Loss: 3.23 	 ROC AUC: 0.65
Epoch: 60 	 Loss: 3.12 	 ROC AUC: 0.67
Epoch: 70 	 Loss: 3.06 	 ROC AUC: 0.68
Epoch: 80 	 Loss: 3.03 	 ROC AUC: 0.69
Epoch: 90 	 Loss: 3.00 	 ROC AUC: 0.68
Epoch: 100 	 Loss: 2.98 	 ROC AUC: 0.69
Epoch: 110 	 Loss: 2.97 	 ROC AUC: 0.68
Epoch: 120 	 Loss: 2.96 	 ROC AUC: 0.68
Epoch: 130 	 Loss: 2.94 	 ROC AUC: 0.69
Epoch: 140 	 Loss: 2.96 	 ROC AUC: 0.70
Epoch: 150 	 Loss: 2.89 	 ROC AUC: 0.70
Epoch: 160 	 Loss: 2.90 	 ROC AUC: 0.71
Epoch: 170 	 Loss: 2.84 	 ROC AUC: 0.71
Epoch: 180 	 Loss: 2.86 	 ROC AUC: 0.72
Epoch: 190 	 Loss: 2.72 	 ROC AUC: 0.75
Epoch: 200 	 Loss: 2.64 	 ROC AUC: 0.75
Epoch: 210 	 Loss: 2.60 	 ROC AUC: 0.77
Epoch: 220 	 Loss: 2.63 	 ROC AUC: 0.78
Epoch: 230 	 Loss: 2.57 	 ROC AUC: 0.79
Epoch: 240 	 Loss: 2.55 	 ROC AUC: 0.79
Epoch: 250 	 Loss: 2.54 	 ROC AUC: 0.79
Epoch: 26

In [ ]:
# Continuation
epochs = 500
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch+500, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 510 	 Loss: 1.97 	 ROC AUC: 0.92
Epoch: 520 	 Loss: 1.93 	 ROC AUC: 0.92
Epoch: 530 	 Loss: 1.93 	 ROC AUC: 0.93
Epoch: 540 	 Loss: 1.94 	 ROC AUC: 0.93
Epoch: 550 	 Loss: 1.91 	 ROC AUC: 0.93
Epoch: 560 	 Loss: 1.85 	 ROC AUC: 0.94
Epoch: 570 	 Loss: 1.88 	 ROC AUC: 0.93
Epoch: 580 	 Loss: 1.89 	 ROC AUC: 0.93
Epoch: 590 	 Loss: 1.90 	 ROC AUC: 0.94
Epoch: 600 	 Loss: 1.84 	 ROC AUC: 0.94
Epoch: 610 	 Loss: 1.86 	 ROC AUC: 0.94
Epoch: 620 	 Loss: 1.88 	 ROC AUC: 0.94
Epoch: 630 	 Loss: 1.89 	 ROC AUC: 0.94
Epoch: 640 	 Loss: 1.82 	 ROC AUC: 0.94
Epoch: 650 	 Loss: 1.85 	 ROC AUC: 0.94
Epoch: 660 	 Loss: 1.81 	 ROC AUC: 0.94
Epoch: 670 	 Loss: 1.93 	 ROC AUC: 0.95
Epoch: 680 	 Loss: 1.84 	 ROC AUC: 0.95
Epoch: 690 	 Loss: 1.87 	 ROC AUC: 0.94
Epoch: 700 	 Loss: 1.80 	 ROC AUC: 0.94
Epoch: 710 	 Loss: 1.78 	 ROC AUC: 0.94
Epoch: 720 	 Loss: 1.77 	 ROC AUC: 0.95
Epoch: 730 	 Loss: 1.82 	 ROC AUC: 0.94
Epoch: 740 	 Loss: 1.81 	 ROC AUC: 0.95
Epoch: 750 	 Loss: 1.80 	 ROC AUC: 0.94


### Dataset: letter

In [ ]:
# letter :: k=40 :: norm=false
dim_h = graph.x.shape[1]
model = GAE(graph.x.shape[1], dim_h, graph.x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

best_roc = 0
epochs = 1000
for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    embeddings = model(graph)
    loss = loss_function(embeddings, graph)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            x_emb = model(graph)
            x_emb = x_emb.detach().numpy()
            loss = loss.detach().numpy()
            roc_auc = evaluate(x, x_emb, y, plot = False)      
            print('Epoch: %2d \t Loss: %.2f \t ROC AUC: %.2f' % (epoch, loss, roc_auc))
            best_roc = max(best_roc, roc_auc)
print('\nROC: %.2f' % best_roc)

Epoch: 10 	 Loss: 18.73 	 ROC AUC: 0.39
Epoch: 20 	 Loss: 13.60 	 ROC AUC: 0.46
Epoch: 30 	 Loss: 12.22 	 ROC AUC: 0.66
Epoch: 40 	 Loss: 11.72 	 ROC AUC: 0.76
Epoch: 50 	 Loss: 10.35 	 ROC AUC: 0.75
Epoch: 60 	 Loss: 10.09 	 ROC AUC: 0.77
Epoch: 70 	 Loss: 9.67 	 ROC AUC: 0.81
Epoch: 80 	 Loss: 9.48 	 ROC AUC: 0.83
Epoch: 90 	 Loss: 9.40 	 ROC AUC: 0.85
Epoch: 100 	 Loss: 8.63 	 ROC AUC: 0.86
Epoch: 110 	 Loss: 8.59 	 ROC AUC: 0.84
Epoch: 120 	 Loss: 8.33 	 ROC AUC: 0.85
Epoch: 130 	 Loss: 8.40 	 ROC AUC: 0.85
Epoch: 140 	 Loss: 8.46 	 ROC AUC: 0.84
Epoch: 150 	 Loss: 8.44 	 ROC AUC: 0.86
Epoch: 160 	 Loss: 8.29 	 ROC AUC: 0.85
Epoch: 170 	 Loss: 8.04 	 ROC AUC: 0.85
Epoch: 180 	 Loss: 8.05 	 ROC AUC: 0.87
Epoch: 190 	 Loss: 7.75 	 ROC AUC: 0.84
Epoch: 200 	 Loss: 7.67 	 ROC AUC: 0.85
Epoch: 210 	 Loss: 7.74 	 ROC AUC: 0.85
Epoch: 220 	 Loss: 7.61 	 ROC AUC: 0.87
Epoch: 230 	 Loss: 7.62 	 ROC AUC: 0.88
Epoch: 240 	 Loss: 8.28 	 ROC AUC: 0.85
Epoch: 250 	 Loss: 7.60 	 ROC AUC: 0.87
Epo